In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://security.ubuntu.com

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

In [3]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/3/DelayedFlights.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("DelayedFlights.csv"), sep=",", header=True)
df.show()

+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
| id|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  0|2008|    1|         3|        4|   2003|      1955|   2211|      2225|       

In [4]:
# Create a temporary view
df.createOrReplaceTempView('delays')

In [5]:
# Run a sql query that groups the data on UniqueCarrier
# note the time functions will track the time it takes to load and run the data
# we are only interested in the time it take to run so run this cell twice.
start_time = time.time()

spark.sql("""select UniqueCarrier,sum(CRSElapsedTime), count(*) from delays group by 1""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+-------------------+--------+
|UniqueCarrier|sum(CRSElapsedTime)|count(1)|
+-------------+-------------------+--------+
|           UA|        1.3998834E7|   82022|
|           AA|        1.7721836E7|  103120|
|           NW|          6761017.0|   48410|
|           EV|          4284049.0|   42782|
|           B6|          4169064.0|   22868|
|           DL|          8245701.0|   48888|
|           OO|          6883377.0|   73680|
|           F9|          2338358.0|   16006|
|           YV|          3216400.0|   34890|
|           US|          8759953.0|   53873|
|           AQ|            99698.0|     750|
|           MQ|          7710479.0|   82505|
|           OH|          3318613.0|   29152|
|           HA|           345580.0|    2597|
|           XE|          7386620.0|   62539|
|           AS|          2527656.0|   16553|
|           FL|          4807695.0|   37201|
|           CO|          8693653.0|   44282|
|           WN|        2.4182455E7|  214624|
|         

In [6]:
# Write out the data in parquet format
df.write.parquet('parquet_delayed', mode='overwrite')

In [7]:
# Read in our new parquet formatted data
p_df=spark.read.parquet('parquet_delayed')

In [8]:
# A parquet formatted DataFrame has all the same methods as a row-based dataframe
# We can convert the dataframe to a view.
p_df.createOrReplaceTempView('p_delays')

In [9]:
start_time = time.time()

spark.sql("""select UniqueCarrier, count(*) from p_delays group by 1""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+--------+
|UniqueCarrier|count(1)|
+-------------+--------+
|           UA|   82022|
|           AA|  103120|
|           NW|   48410|
|           EV|   42782|
|           B6|   22868|
|           DL|   48888|
|           OO|   73680|
|           F9|   16006|
|           YV|   34890|
|           US|   53873|
|           AQ|     750|
|           MQ|   82505|
|           OH|   29152|
|           HA|    2597|
|           XE|   62539|
|           AS|   16553|
|           FL|   37201|
|           CO|   44282|
|           WN|  214624|
|           9E|   31833|
+-------------+--------+

--- 1.7447712421417236 seconds ---


In [10]:
# Here is another sample
start_time = time.time()

spark.sql("""select UniqueCarrier,sum(CRSElapsedTime), count(*) from p_delays group by 1""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+-------------------+--------+
|UniqueCarrier|sum(CRSElapsedTime)|count(1)|
+-------------+-------------------+--------+
|           UA|        1.3998834E7|   82022|
|           AA|        1.7721836E7|  103120|
|           NW|          6761017.0|   48410|
|           EV|          4284049.0|   42782|
|           B6|          4169064.0|   22868|
|           DL|          8245701.0|   48888|
|           OO|          6883377.0|   73680|
|           F9|          2338358.0|   16006|
|           YV|          3216400.0|   34890|
|           US|          8759953.0|   53873|
|           AQ|            99698.0|     750|
|           MQ|          7710479.0|   82505|
|           OH|          3318613.0|   29152|
|           HA|           345580.0|    2597|
|           XE|          7386620.0|   62539|
|           AS|          2527656.0|   16553|
|           FL|          4807695.0|   37201|
|           CO|          8693653.0|   44282|
|           WN|        2.4182455E7|  214624|
|         

In [11]:
# Partition our data by UniqueCarrier
df.write.partitionBy("UniqueCarrier").mode("overwrite").parquet("delayed_partitioned")

In [12]:
# Read in our new parquet formatted data
p_df_p=spark.read.parquet('delayed_partitioned')

In [13]:
# Convert the DataFrame to a view.
p_df_p.createOrReplaceTempView('p_delays_p')

In [14]:
# Query the partitioned data on the Partition key.
start_time = time.time()

spark.sql("""select UniqueCarrier, count(*) from p_delays_p group by 1""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+--------+
|UniqueCarrier|count(1)|
+-------------+--------+
|           UA|   82022|
|           AA|  103120|
|           NW|   48410|
|           DL|   48888|
|           OO|   73680|
|           US|   53873|
|           MQ|   82505|
|           XE|   62539|
|           CO|   44282|
|           WN|  214624|
|           EV|   42782|
|           B6|   22868|
|           F9|   16006|
|           YV|   34890|
|           AQ|     750|
|           OH|   29152|
|           HA|    2597|
|           AS|   16553|
|           FL|   37201|
|           9E|   31833|
+-------------+--------+

--- 1.2399756908416748 seconds ---


In [15]:
# Grouping by partition key and aggregating data.
start_time = time.time()

spark.sql("""select UniqueCarrier,sum(CRSElapsedTime) from p_delays_p group by 1""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+-------------------+
|UniqueCarrier|sum(CRSElapsedTime)|
+-------------+-------------------+
|           UA|        1.3998834E7|
|           AA|        1.7721836E7|
|           NW|          6761017.0|
|           DL|          8245701.0|
|           OO|          6883377.0|
|           US|          8759953.0|
|           MQ|          7710479.0|
|           XE|          7386620.0|
|           CO|          8693653.0|
|           WN|        2.4182455E7|
|           EV|          4284049.0|
|           B6|          4169064.0|
|           F9|          2338358.0|
|           YV|          3216400.0|
|           AQ|            99698.0|
|           OH|          3318613.0|
|           HA|           345580.0|
|           AS|          2527656.0|
|           FL|          4807695.0|
|           9E|          3255692.0|
+-------------+-------------------+

--- 1.8216934204101562 seconds ---


In [16]:
# Another query filtering on the key.
start_time = time.time()
spark.sql("""Select UniqueCarrier, sum(DepDelay) as total_delayed from p_delays_p where UniqueCarrier='US' group by 1""").show()
print("--- %s seconds ---" % (time.time() - start_time))

+-------------+-------------+
|UniqueCarrier|total_delayed|
+-------------+-------------+
|           US|    2077273.0|
+-------------+-------------+

--- 0.5609829425811768 seconds ---


In [17]:
# Same query as above against the parquet (non-partitioned) data.
start_time = time.time()
spark.sql("""Select UniqueCarrier, sum(DepDelay) as total_delayed from p_delays where UniqueCarrier='US' group by 1""").show()
print("--- %s seconds ---" % (time.time() - start_time))

+-------------+-------------+
|UniqueCarrier|total_delayed|
+-------------+-------------+
|           US|    2077273.0|
+-------------+-------------+

--- 0.8344180583953857 seconds ---


In [18]:
# Here is a query that doesn't use the partition key at all (against the parquet data)
start_time = time.time()
spark.sql("""Select distinct UniqueCarrier, TailNum from p_delays where TailNum='N712SW' """).show()
print("--- %s seconds ---" % (time.time() - start_time))

+-------------+-------+
|UniqueCarrier|TailNum|
+-------------+-------+
|           WN| N712SW|
+-------------+-------+

--- 0.8952834606170654 seconds ---


In [19]:
# Here is a query that doesn't use the partition key at all (against the partitioned parquet data)
start_time = time.time()
spark.sql("""Select distinct UniqueCarrier, TailNum from p_delays_p where TailNum='N712SW' """).show()
print("--- %s seconds ---" % (time.time() - start_time))

+-------------+-------+
|UniqueCarrier|TailNum|
+-------------+-------+
|           WN| N712SW|
+-------------+-------+

--- 0.7348916530609131 seconds ---
